# Anime Generation
## DCGAN 

> Data pre-processing [here](https://www.tensorflow.org/tutorials/load_data/images)

In [ ]:
import time
import os

# trainlets < version 5.0.0
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# os.environ['WANDB_NOTEBOOK_NAME'] = 'Anime Faces and Names'
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# trainlets > version 5.0.0 required for tensorflow 2.2.1 and python 3.9
os.environ.get('TF_GPU_ALLOCATOR', 'cuda_malloc_async')
# os.environ.get('WANDB_NOTEBOOK_NAME', "Anime Faces and Names")
os.environ.get('TF_CPP_MIN_LOG_LEVEL', '2')

import winsound
from IPython import display
the_voice = "./sounds/chose_a_voice.wav"
the_creation = "./sounds/You_created_me.wav"

import tensorflow as tf
# Error: Node: 'sequential_3/dropout_4/dropout/random_uniform/RandomUniform'
# OOM when allocating tensor with shape[256,180,120,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
# 	 [[{{node sequential_3/dropout_4/dropout/random_uniform/RandomUniform}}]]
# the solution to thi is report_tensor_allocations_upon_oom = True
# Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. 
# This isn't available when running in Eager mode.
# In this project we are running in eager mode!
# you can check this by running the following code: tf.executing_eagerly()
# run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom=True)
# print("run_opts : => ", run_opts)

# runmeta = tf.compat.v1.RunMetadata()
# physical_devices = tf.config.list_physical_devices("GPU")
# print("physical_devices : => ", physical_devices)
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.distribute.MultiWorkerMirroredStrategy(
#     cluster_resolver=None, communication_options=None
# )

from tensorflow import keras
from tensorflow.keras import layers, preprocessing
from tensorflow.keras.optimizers import Adam
import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import tensorflow_docs
import PIL

print("Note! You will need tensorflow-gpu version 2.8.0 or higher. Your version is", tf.__version__)

winsound.PlaySound(sound=the_voice, flags=winsound.SND_ALIAS)

> Configuration

In [ ]:
# BATCH_SIZE = 254 # or 64 this cause memory allocation failure
BATCH_SIZE = 64
BUFFER_SIZE = 2000
IMAGE_WIDTH = 240
IMAGE_LENGTH = 360
ORIGINAL_IMAGE_SIZE = (360, 240)
EPOCHS = 15000
CURRENT_EPOCH = 3000
noise_dim = 100
num_examples_to_generate = 9
learning_rate=0.0002
# data_directory= "./dataset_faces/"
data_directory= "./dataset/"
anim_file = "anime_characters_dcgan.gif"
# folder = './anime_faces/'
folder = "./new_anime_chars/"
new_images_root = folder + "generated_images/"
training_checkpoints_root = folder + "training_checkpoints/"

# global variables
global genereator_loss_list, discriminator_loss_list 
genereator_loss_list, discriminator_loss_list = [], []

# Load Dataset

In [ ]:
%%time

x_train = preprocessing.image_dataset_from_directory( 
  directory= data_directory, 
  validation_split=0.5,
  subset='training',
  label_mode=None, 
  batch_size=BATCH_SIZE, 
  image_size=ORIGINAL_IMAGE_SIZE, 
  seed=123 , 
shuffle=True)

plt.figure(figsize=(10, 10))
plt.title("Training Images")
for images in x_train.take(1):
  print(images.shape)
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis("off")


> Cashe Dataset

In [ ]:
%%time
print("Wall time: 5min 22s")
from os.path import exists

file_exists = exists('lost_list.csv') and os.stat('lost_list.csv').st_size == 0
if file_exists:
    data_loss = pd.read_csv('lost_list.csv')
    global genereator_loss_list, discriminator_loss_list
    genereator_loss_list, discriminator_loss_list = data_loss['genereator_loss_list'].to_numpy().tolist(), data_loss['discriminator_loss_list'].to_numpy()
    
# print("genereator_loss_list = ", genereator_loss_list)
# print("discriminator_loss_list = ", discriminator_loss_list)

# Cache keeps the images in memory after they're loaded off disk during the first epoch. 
# This will ensure the dataset does not become a bottleneck while training your model. 
# Prefetch overlaps data preprocessing and model execution while training.
# This allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
def configure_for_performance(ds, name):
    ds = ds.cache(name)
    # ds = ds.shuffle(buffer_size=2000) # we are making shuffle at the loading
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

# normalized_ds = x_train.map(tf.keras.layers.Rescaling(scale=1./255)) # scale 0 to 1
normalized_ds = x_train.map(tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)) # scale -1 to 1 because we are using tanh
# Cashe, shuffle and prefetch the data
train_ds = configure_for_performance(normalized_ds, "./cache/training_cashe")

for image_batch in train_ds:
    print(image_batch.shape)
    print("In range between: ", np.min(image_batch[0])," and: ", np.max(image_batch[0]))
    break


## DCGAN 
> Generator

In [ ]:
def generator_model():
    print("Generating DCGAN model.")
    model = tf.keras.Sequential()
    model.add(layers.Dense(45*30*256, use_bias=False, input_shape=(100,)))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Reshape((45, 30, 256)))
    assert model.output_shape == (None, 45, 30, 256)  # Note: None is the batch size
    
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 45, 30, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 90, 60, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    
    # Add another Conv2DTranspose to convert the image to the original dimensions
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 90, 60, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 180, 120, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 360, 240, 3)

    print("DCGAN model completed")
    return model

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
%%time
noise = tf.random.normal([BATCH_SIZE, noise_dim])
generator = generator_model()
generated_image = generator(noise, training=False)
print(generated_image.shape)
plt.imshow(generated_image[0, :, :, 0]) # default RGB
generator.summary()

> Discriminator 

In [ ]:
def discriminator_model():
    print("DCGAN descriminator model")
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                    input_shape=[360, 240, 3]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(5, 5)))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(5, 5)))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1))
    
    print("DCGAN descriminator Completed!!")
    return model

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
discriminator = discriminator_model()
decision = discriminator(generated_image)
# print(decision)
discriminator.summary()

### Save and plot images

### Optimizer and Checkpoint 

In [ ]:
def read_images(folder):
    filenames = glob.glob(new_images_root + folder + 'image*.png')
    return sorted(filenames)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    # predictions = ((predictions + 1) /2)*255
    
    for i in range(predictions.shape[0]):
        img = tf.keras.preprocessing.image.array_to_img(
                (predictions[i] + 1) * 127.5, scale=False
                )
        name = 'image_at_epoch_{:04d}.png'.format(i)
        img.save(os.path.join(new_images_root + 'steps', name))
    
    file_images = read_images('steps/')
    fig = plt.figure(figsize=(10, 10))
    i=0
    for imgs in file_images:
        image = imageio.imread(imgs)
        plt.subplot(3, 3, i+1)
        plt.imshow(image)
        plt.axis('off')
        i+=1
    plt.savefig(new_images_root + 'image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    

In [ ]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

generator_optimizer = Adam(learning_rate)
discriminator_optimizer = Adam(learning_rate)

checkpoint_dir = training_checkpoints_root
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    generator=generator,
    discriminator=discriminator)


> Plot loss

In [ ]:
# PLOT
def plot_loss(gen_loss, disc_loss):
    x = np.arange(0, len(gen_loss) )

    # plotting
    fig_gen = plt.figure(figsize=(10, 6), dpi=80)
    plt.title("Generator Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.plot(x, gen_loss, color ="green")
    plt.show()
    fig_gen.savefig(new_images_root + 'gen_loss_plot.png')
    plt.close()

    fig_disc = plt.figure(figsize=(10, 6), dpi=80)
    plt.title("Discriminator Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.plot(x, disc_loss, color ="green")
    plt.show()
    fig_disc.savefig(new_images_root + 'disc_loss_plot.png')
    plt.close()

    fig_gen_disc = plt.figure(figsize=(10, 6), dpi=80)
    plt.title("Generator and Discriminator Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.plot(x, gen_loss, color ="green")
    plt.plot(x, disc_loss, color ="red")
    plt.show()
    fig_gen_disc.savefig(new_images_root + 'loss_plot.png')
    plt.close()

def save_loss_csv(genereator_loss, discriminator_loss):
    data_loss = pd.DataFrame({"genereator_loss_list" : genereator_loss, "discriminator_loss_list" : discriminator_loss})
    data_loss.to_csv("lost_list.csv", index=False)

## Training DCGAN

### Training definitions 

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        generated_images = generator(noise, training=True)
        real_out = discriminator(images, training=True)
        fake_out = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_out)
        
        disc_loss = discriminator_loss(real_output=real_out, fake_output=fake_out)
        
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        
        return gen_loss, disc_loss, generated_image


In [ ]:
def train(dataset, epochs):
    print("Training Start...")
    global genereator_loss_list, discriminator_loss_list
    
    for epoch in range(CURRENT_EPOCH, epochs):
        print("Epoch :", epoch, "/", EPOCHS)
        start = time.time()
        start_step_time = start

        for step, image_batch in enumerate(dataset):
            gen_loss, disc_loss, generated_image = train_step(image_batch)
            if step % 100 == 0:
                print('step: %d - gen_loss: %.4f - disc_loss: %.4f - %.3f s' % (step, gen_loss, disc_loss, time.time()-start_step_time))
                start_step_time = time.time()
        
        genereator_loss_list = np.append(genereator_loss_list, gen_loss.numpy())
        discriminator_loss_list = np.append(discriminator_loss_list, disc_loss.numpy())
        
        display.clear_output(wait=True)
        # Save the model and image every 5 epochs
        if (epoch + 1) % 10 == 0:
            # Produce images for the GIF as you go
            generate_and_save_images(generator, epoch + 1, seed)
            save_loss_csv(genereator_loss_list, discriminator_loss_list)
        if (epoch + 1) % 50 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            plot_loss(genereator_loss_list, discriminator_loss_list)
            winsound.Beep(frequency=1000, duration=500 )

        print ('Time for epoch {} is {} sec'.format(epoch, time.time()-start))

    # Generate image after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)


### Start Training

In [ ]:
%%time
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
train(dataset=train_ds, epochs=EPOCHS)

winsound.PlaySound(sound=the_creation, flags=winsound.SND_ALIAS)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

 ## Create a GIF

In [ ]:
%%time
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open(new_images_root + 'image_at_epoch_{:04d}.png'.format(epoch_no))
display_image(EPOCHS)


In [ ]:
with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = read_images('')
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
    # Added this to make the gif more "proper"
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
print(discriminator_loss_list)
print(genereator_loss_list)

save_loss_csv(genereator_loss_list, discriminator_loss_list)

In [ ]:
plot_loss(genereator_loss_list, discriminator_loss_list)